In [ ]:
pip install qwen_vl_utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 62.5 MB/s eta 0:00:00


In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image

# ... (rest of your imports and model loading code) ...

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",  # Replace with your image path
                "resized_height": 280,  # Adjust this value as needed
                "resized_width": 420,  # Adjust this value as needed
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)

# If your image path is a local file, you need to open it with PIL and resize it
for i, image_path in enumerate(image_inputs):
    if isinstance(image_path, str) and image_path.startswith("file:///"):
        image = Image.open(image_path[7:])  # Remove "file:///" prefix
        image = image.resize((messages[0]["content"][0]["resized_width"], messages[0]["content"][0]["resized_height"]))
        image_inputs[i] = image

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
# ... (rest of your code) ...

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import torch
import requests
from io import BytesIO

# Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)

# Load the processor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# Image URL
image_path = "/content/下載.jpg"
image = Image.open(image_path)

# Resize parameters (change these values as needed)
resized_width = 420
resized_height = 280

# Download the image from URL
# response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Resize the image to the required dimensions
image = image.resize((resized_width, resized_height))

# Example message structure with resized image
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image,  # Pass resized image
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

# Process vision information (image and video)
image_inputs, video_inputs = process_vision_info(messages)

# Prepare the inputs for the model
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

# Move inputs to the appropriate device (GPU)
inputs = inputs.to("cuda")

# Inference: Generate the output
generated_ids = model.generate(**inputs, max_new_tokens=128)

# Trim the generated IDs to align with the input size (ignore prompt tokens)
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

# Decode the generated output into text
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# Print the generated output text
print(output_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['The image depicts a young woman sitting on a sandy beach with her dog. The dog is sitting on its hind legs, and the woman is petting it. The background shows the ocean and a clear sky, suggesting a serene and peaceful setting. The woman appears to be smiling, indicating a happy and joyful moment.']


In [ ]:
pip install transformers Pillow


In [ ]:
pip install qwen_vl_utils


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 64.5 MB/s eta 0:00:00


In [ ]:
from google.colab import files
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import torch

# 載入模型與處理器
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# 定義 min_pixels 和 max_pixels (optional)
min_pixels = 280 * 280  # 例如：280x280 的圖片像素
max_pixels = 420 * 420  # 例如：420x420 的圖片像素
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels
)

# 讓使用者從本地電腦上傳檔案
print("請選擇圖片檔案進行上傳...")
uploaded = files.upload()  # 使用者上傳檔案

# 取得上傳檔案的路徑
for file_name in uploaded.keys():
    image_path = file_name

# 開啟並調整圖片大小
image = Image.open(image_path)

# 第一種方式：直接指定 resized_height 和 resized_width
resized_height = 280
resized_width = 420
image_resized = image.resize((resized_width, resized_height))

# 第二種方式：利用 min_pixels 和 max_pixels
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_resized,  # 使用已調整大小的圖片
                "min_pixels": min_pixels,
                "max_pixels": max_pixels,
            },
            {"type": "text", "text": "描述這張圖片中的食物項目，並提供它們的 approximate 熱量內容。"
},
        ],
    }
]

# 準備推論輸入
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)

inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

# 移動到 GPU
inputs = inputs.to("cuda")

# 模型推論
generated_ids = model.generate(**inputs, max_new_tokens=128)

# 去除多餘的輸出內容（只保留模型生成的部分）
generated_ids_trimmed = [
    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

# 將生成的輸出轉換為文字
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# 輸出結果
print("生成的描述為：")
print(output_text[0])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

請選擇圖片檔案進行上傳...


Saving breakfast.jpg to breakfast (7).jpg
生成的描述為：
這張圖片展示了一個早餐盤，包含以下食物項目：

1. 煎蛋：煎蛋的熱量約為120卡路里。
2. 烤面包：烤面包的熱量約為100卡路里。
3. 草莓：草莓的熱量約為50卡路里。
4. 蓝莓：藍莓的熱量約為50卡路里。
5. 咖啡：咖啡的熱量約為100卡路里。

總熱量：約為370卡路里。


In [ ]:
!pip install googletrans==4.0.0-rc1


In [ ]:
from google.colab import files
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import torch
from googletrans import Translator  # 引入翻譯工具

# 載入模型與處理器
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# 模擬 process_vision_info 功能
def process_vision_info(messages):
    image_inputs = []
    video_inputs = []
    for message in messages:
        for content in message["content"]:
            if content["type"] == "image":
                image_inputs.append(content["image"])
            elif content["type"] == "video":
                video_inputs.append(content["video"])
    return image_inputs, video_inputs

# 讓使用者從本地電腦上傳檔案
print("請選擇圖片檔案進行上傳...")
uploaded = files.upload()

# 開啟上傳的檔案
for file_name in uploaded.keys():
    image_path = file_name
image = Image.open(image_path)

# 調整圖片大小
resized_width = 420
resized_height = 280
image_resized = image.resize((resized_width, resized_height))

# 建立消息
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_resized},
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# 準備輸入
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

# 將輸入移動至 GPU
inputs = inputs.to("cuda")

# 推論
generated_ids = model.generate(**inputs, max_new_tokens=128)
output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# 使用翻譯工具將英文轉換為中文
translator = Translator()
translated_text = translator.translate(output_text[0], src="en", dest="zh-tw").text

# 輸出結果
print("生成的描述為（英文）：")
print(output_text[0])
print("\n生成的描述為（中文）：")
print(translated_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

請選擇圖片檔案進行上傳...


Saving breakfast.jpg to breakfast (1).jpg


IndexError: list index out of range

In [ ]:
from google.colab import files
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import torch

# 載入模型與處理器
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# 模擬 process_vision_info 功能
def process_vision_info(messages):
    image_inputs = []
    video_inputs = []
    for message in messages:
        for content in message["content"]:
            if content["type"] == "image":
                image_inputs.append(content["image"])
    return image_inputs, video_inputs

# 讓使用者從本地電腦上傳檔案
print("請選擇圖片檔案進行上傳...")
uploaded = files.upload()

# 開啟上傳的檔案
for file_name in uploaded.keys():
    image_path = file_name
image = Image.open(image_path)

# 調整圖片大小
resized_width = 420
resized_height = 280
image_resized = image.resize((resized_width, resized_height))

# 建立消息
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_resized},
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# 檢查消息內容
print("Messages:", messages)

# 準備輸入
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)

# 檢查圖像輸入是否正確
print("Image Inputs:", image_inputs)
print("Video Inputs:", video_inputs)

# 準備輸入並移動至 GPU
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# 推論
generated_ids = model.generate(**inputs, max_new_tokens=128)
output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# 輸出結果
print("生成的描述為：")
print(output_text[0])


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

請選擇圖片檔案進行上傳...


Saving breakfast.jpg to breakfast (2).jpg
Messages: [{'role': 'user', 'content': [{'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=420x280 at 0x7EFB0A9E70A0>}, {'type': 'text', 'text': 'Describe this image.'}]}]
Image Inputs: [<PIL.Image.Image image mode=RGB size=420x280 at 0x7EFB0A9E70A0>]
Video Inputs: []


IndexError: list index out of range